In [21]:
pip install whisperx

In [22]:
# Cài transformers để chỉnh chính tả
!pip install transformers accelerate --quiet

In [23]:
def frame_to_seconds(frame_number, fps):
    return frame_number / fps

In [24]:
import os

SHOTS_PATH = "/content/L01_V007_shots.txt"
FPS = 25.0  # Hoặc FPS thực tế

keyframe_intervals = []
with open(SHOTS_PATH, "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        start_frame, end_frame = map(int, line.strip().split())
        start_sec = frame_to_seconds(start_frame, FPS)
        end_sec = frame_to_seconds(end_frame, FPS)
        keyframe_intervals.append((start_sec, end_sec))

print("Tổng đoạn keyframe:", len(keyframe_intervals))

Tổng đoạn keyframe: 299


In [25]:
import torch
import whisperx

VIDEO_PATH = "/content/L01_V007.mp4"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = whisperx.load_model("medium", device=device, compute_type="int8")
audio = whisperx.load_audio(VIDEO_PATH)
result = model.transcribe(audio)

model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
aligned_result = whisperx.align(result["segments"], model_a, metadata, audio, device=device)

print("Tổng số đoạn whisper:", len(aligned_result["segments"]))


No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.
Detected language: vi (1.00) in first 30s of audio...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at nguyenvulebinh/wav2vec2-base-vi and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tổng số đoạn whisper: 152


In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("bmd1905/vietnamese-correction")
corrector = AutoModelForSeq2SeqLM.from_pretrained("bmd1905/vietnamese-correction").to(device)

def correct_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = corrector.generate(**inputs, max_new_tokens=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [27]:
from transformers import pipeline

# Khởi tạo pipeline sửa chính tả
corrector = pipeline("text2text-generation", model="bmd1905/vietnamese-correction")

# Hàm sửa văn bản
def correct_text(text):
    return corrector(text, max_length=512, do_sample=False)[0]["generated_text"]

# Gộp các segment theo keyframe rồi sửa lỗi
new_segments = []
seg_idx = 0

for start_kf, end_kf in keyframe_intervals:
    merged_text = []

    # Duyệt các đoạn transcript có trong khoảng keyframe
    while seg_idx < len(aligned_result["segments"]):
        seg = aligned_result["segments"][seg_idx]
        if seg["end"] <= start_kf:
            seg_idx += 1
            continue
        if seg["start"] >= end_kf:
            break
        merged_text.append(seg["text"])
        seg_idx += 1

    # Nếu có đoạn để gộp thì tiến hành sửa chính tả
    if merged_text:
        raw_text = " ".join(merged_text)
        corrected_text = correct_text(raw_text)
        new_segments.append({
            "start": start_kf,
            "end": end_kf,
            "text": corrected_text
        })

print("Số đoạn mới:", len(new_segments))

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_cla

Số đoạn mới: 117


In [28]:
for i, seg in enumerate(new_segments):
    print(f"{seg['start']:.2f}s → {seg['end']:.2f}s: {seg['text']}")

0.00s → 0.64s: Cái chào quý khán giả Hãy theo dõi chương trình Sống ở dây sáng Chủ nhật ngày 8 tháng 6 của đài truyền hình Thành phố Hồ Chí Minh Với nhiều thông tin cập nhật trong đêm, và bây giờ sẽ là những nội dung chính có trong chương trình Sở Giáo dục đào tạo Thành phố Hồ Chí Minh thông báo thời gian.. Chự kiến công bố điểm thi đấp 10 bé 15 tháng tuổi hồi sinh sau ca ghép gàng.
31.80s → 32.52s: Đêm qua, lửa bao trừng nhà hàng trong khu công nghiệp sống thần.
32.56s → 34.36s: Và ngay sau đây là phần nội dung chi tiết. Tối qua, Sở Giáo dục đào tạo TP.HCM đã công bố các mốc thời gian quan trọng sau kỳ thi tuyển sinh lớp 10.
35.64s → 37.16s: Theo đó, công tác chấm thi lớp 10 sẽ kết thúc vào ngày 16 tháng 6 và đến 23 tháng 10 thì dự kiến sẽ công bố kết quả.
55.32s → 57.32s: Từ ngày 18 đến ngày 20 tháng 6, hội phách đối sánh kết quả về thi với kết quả trên máy tính.
58.92s → 60.60s: Từ ngày 21 đến 22 tháng 6, đồng bộ kết quả thi về cơ sở dữ liệu của ngành.
60.64s → 65.20s: Từ ngày 23 đế

In [29]:
import json
print(json.dumps(new_segments, ensure_ascii=False, indent=4))

[
    {
        "start": 0.0,
        "end": 0.64,
        "text": "Cái chào quý khán giả Hãy theo dõi chương trình Sống ở dây sáng Chủ nhật ngày 8 tháng 6 của đài truyền hình Thành phố Hồ Chí Minh Với nhiều thông tin cập nhật trong đêm, và bây giờ sẽ là những nội dung chính có trong chương trình Sở Giáo dục đào tạo Thành phố Hồ Chí Minh thông báo thời gian.. Chự kiến công bố điểm thi đấp 10 bé 15 tháng tuổi hồi sinh sau ca ghép gàng."
    },
    {
        "start": 31.8,
        "end": 32.52,
        "text": "Đêm qua, lửa bao trừng nhà hàng trong khu công nghiệp sống thần."
    },
    {
        "start": 32.56,
        "end": 34.36,
        "text": "Và ngay sau đây là phần nội dung chi tiết. Tối qua, Sở Giáo dục đào tạo TP.HCM đã công bố các mốc thời gian quan trọng sau kỳ thi tuyển sinh lớp 10."
    },
    {
        "start": 35.64,
        "end": 37.16,
        "text": "Theo đó, công tác chấm thi lớp 10 sẽ kết thúc vào ngày 16 tháng 6 và đến 23 tháng 10 thì dự kiến sẽ công bố kết quả."

In [30]:
with open("L01_V007_script.json", "w", encoding="utf-8") as f:
    json.dump(new_segments, f, ensure_ascii=False, indent=4)